In [1]:
import pickle, os, json
import numpy as np
import matplotlib.pyplot as plt

from qcutils.pulse.amptimeseries.metric import dist

In [2]:
def read_data(filepath):
    with open(filepath, "rb") as f:
        power_timeseries_list = pickle.load(f)
    return power_timeseries_list

In [3]:
def compute_accuracy(filepath, add_error = False, error_rate = 0, random_seed = None):
    if random_seed is not None:
        np.random.seed(random_seed)

    power_timeseries_list = read_data(filepath)
    energy_list = [np.sum(power_timeseries) for power_timeseries in power_timeseries_list]
    mean_power_list = [np.mean(power_timeseries) for power_timeseries in power_timeseries_list]
    total_duration_list = [len(power_timeseries) for power_timeseries in power_timeseries_list]

    def identification_accuracy(metric_list, add_error = False, error_rate = 0):
        total_exp_num = len(metric_list)
        per_cent_num = int(total_exp_num / 100)

        count = 0
        for i, metric in enumerate(metric_list):
            if add_error:
                if not isinstance(metric, np.ndarray):
                    current_metric = metric * (1 + np.random.normal(loc=0, scale=error_rate, size=1).item())
                else:
                    current_metric = metric * (1 + np.random.normal(loc=0, scale=error_rate, size=len(metric)))
            else:
                current_metric = metric
            dist_list = dist(current_metric, metric_list)
            guess = np.argmin(dist_list)
            count += i == guess

        #     try:
        #         if i % per_cent_num == 0:
        #             progress_bar(i, total_exp_num)
        #     except:
        #         continue

        # import sys
        # sys.stdout.write('\n')

        return count/total_exp_num
    
    power_timeseries_acc = identification_accuracy(power_timeseries_list, add_error=add_error, error_rate=error_rate)
    energy_acc = identification_accuracy(energy_list, add_error=add_error, error_rate=error_rate)
    mean_power_acc = identification_accuracy(mean_power_list, add_error=add_error, error_rate=error_rate)
    duration_acc = identification_accuracy(total_duration_list, add_error=add_error, error_rate=error_rate)

    return (power_timeseries_acc, energy_acc, mean_power_acc, duration_acc)

In [4]:
def save_result(filepath_list, add_error = False, error_rate = 0, do_print = True, save_filepath = None):
    accuracy = {"error_rate": error_rate, "accuracy": {"power_timeseries": [], "energy": [], "mean_power": [], "duration": []}}
    for filepath in filepath_list:
        print(f"File: {filepath}")

        power_timeseries_acc, energy_acc, mean_power_acc, duration_acc = compute_accuracy(filepath, add_error=add_error, error_rate=error_rate)
        accuracy["accuracy"]["power_timeseries"].append(power_timeseries_acc)
        accuracy["accuracy"]["energy"].append(energy_acc)
        accuracy["accuracy"]["mean_power"].append(mean_power_acc)
        accuracy["accuracy"]["duration"].append(duration_acc)

        if do_print:
            print(f"Power Timeseries: {power_timeseries_acc}")
            print(f"Total Energy: {energy_acc}")
            print(f"Mean Power: {mean_power_acc}")
            print(f"Duration: {duration_acc}")
            print("\n")
    
    if save_filepath:
        with open(save_filepath, "w") as f:
            json.dump(accuracy, f, indent=4)

In [6]:
add_error = False
error_rate = 0

num_layout_list = [2**i for i in range(8)]
filepath_list = [os.path.join(os.getcwd(), "data", f"layout_{num_layout}.pickle") for num_layout in num_layout_list]

save_result(filepath_list, add_error=add_error, error_rate=error_rate, do_print=True, save_filepath=os.path.join(os.getcwd(), "data", "accuracy", f"error_rate_{str(error_rate)}.json"))

File: e:\Research\quantum-computing-power-side-channel-attacks\experiments\user_circuit_identification\data\layout_1.pickle
Power Timeseries: 1.0
Total Energy: 1.0
Mean Power: 1.0
Duration: 1.0


File: e:\Research\quantum-computing-power-side-channel-attacks\experiments\user_circuit_identification\data\layout_2.pickle
Power Timeseries: 1.0
Total Energy: 1.0
Mean Power: 1.0
Duration: 0.9516129032258065


File: e:\Research\quantum-computing-power-side-channel-attacks\experiments\user_circuit_identification\data\layout_4.pickle
Power Timeseries: 1.0
Total Energy: 1.0
Mean Power: 0.9838709677419355
Duration: 0.9112903225806451


File: e:\Research\quantum-computing-power-side-channel-attacks\experiments\user_circuit_identification\data\layout_8.pickle
Power Timeseries: 0.9798387096774194
Total Energy: 0.9758064516129032
Mean Power: 0.967741935483871
Duration: 0.8346774193548387


File: e:\Research\quantum-computing-power-side-channel-attacks\experiments\user_circuit_identification\data\layo

In [6]:
add_error = True
error_rate = 1e-5

num_layout_list = [2**i for i in range(8)]
filepath_list = [os.path.join(os.getcwd(), "data", f"layout_{num_layout}.pickle") for num_layout in num_layout_list]

save_result(filepath_list, add_error=add_error, error_rate=error_rate, do_print=True, save_filepath=os.path.join(os.getcwd(), "data", "accuracy", f"error_rate_{str(error_rate)}.json"))

File: e:\Research\quantum-computing-power-side-channel-attacks\experiments\user_circuit_identification\data\layout_1.pickle
Power Timeseries: 1.0
Total Energy: 1.0
Mean Power: 1.0
Duration: 1.0


File: e:\Research\quantum-computing-power-side-channel-attacks\experiments\user_circuit_identification\data\layout_2.pickle
Power Timeseries: 1.0
Total Energy: 1.0
Mean Power: 0.9838709677419355
Duration: 0.9516129032258065


File: e:\Research\quantum-computing-power-side-channel-attacks\experiments\user_circuit_identification\data\layout_4.pickle
Power Timeseries: 1.0
Total Energy: 0.9838709677419355
Mean Power: 0.9838709677419355
Duration: 0.9112903225806451


File: e:\Research\quantum-computing-power-side-channel-attacks\experiments\user_circuit_identification\data\layout_8.pickle
Power Timeseries: 0.9717741935483871
Total Energy: 0.9758064516129032
Mean Power: 0.9637096774193549
Duration: 0.8346774193548387


File: e:\Research\quantum-computing-power-side-channel-attacks\experiments\user_c

In [5]:
add_error = True
error_rate = 1e-4

num_layout_list = [2**i for i in range(8)]
filepath_list = [os.path.join(os.getcwd(), "data", f"layout_{num_layout}.pickle") for num_layout in num_layout_list]

save_result(filepath_list, add_error=add_error, error_rate=error_rate, do_print=True, save_filepath=os.path.join(os.getcwd(), "data", "accuracy", f"error_rate_{str(error_rate)}.json"))

File: e:\Research\quantum-computing-power-side-channel-attacks\experiments\user_circuit_identification\data\layout_1.pickle
Power Timeseries: 1.0
Total Energy: 1.0
Mean Power: 1.0
Duration: 1.0


File: e:\Research\quantum-computing-power-side-channel-attacks\experiments\user_circuit_identification\data\layout_2.pickle
Power Timeseries: 1.0
Total Energy: 1.0
Mean Power: 0.967741935483871
Duration: 0.9516129032258065


File: e:\Research\quantum-computing-power-side-channel-attacks\experiments\user_circuit_identification\data\layout_4.pickle
Power Timeseries: 1.0
Total Energy: 1.0
Mean Power: 0.9838709677419355
Duration: 0.9112903225806451


File: e:\Research\quantum-computing-power-side-channel-attacks\experiments\user_circuit_identification\data\layout_8.pickle
Power Timeseries: 0.9758064516129032
Total Energy: 0.9637096774193549
Mean Power: 0.8991935483870968
Duration: 0.8346774193548387


File: e:\Research\quantum-computing-power-side-channel-attacks\experiments\user_circuit_identific

In [5]:
add_error = True
error_rate = 1e-3

num_layout_list = [2**i for i in range(8)]
filepath_list = [os.path.join(os.getcwd(), "data", f"layout_{num_layout}.pickle") for num_layout in num_layout_list]

save_result(filepath_list, add_error=add_error, error_rate=error_rate, do_print=True, save_filepath=os.path.join(os.getcwd(), "data", "accuracy", f"error_rate_{str(error_rate)}.json"))

File: e:\Research\quantum-computing-power-side-channel-attacks\experiments\user_circuit_identification\data\layout_1.pickle
Power Timeseries: 1.0
Total Energy: 1.0
Mean Power: 0.9032258064516129
Duration: 1.0


File: e:\Research\quantum-computing-power-side-channel-attacks\experiments\user_circuit_identification\data\layout_2.pickle
Power Timeseries: 1.0
Total Energy: 0.9516129032258065
Mean Power: 0.9193548387096774
Duration: 0.9516129032258065


File: e:\Research\quantum-computing-power-side-channel-attacks\experiments\user_circuit_identification\data\layout_4.pickle
Power Timeseries: 1.0
Total Energy: 0.9274193548387096
Mean Power: 0.8064516129032258
Duration: 0.9032258064516129


File: e:\Research\quantum-computing-power-side-channel-attacks\experiments\user_circuit_identification\data\layout_8.pickle
Power Timeseries: 0.9758064516129032
Total Energy: 0.875
Mean Power: 0.6330645161290323
Duration: 0.7943548387096774


File: e:\Research\quantum-computing-power-side-channel-attacks\e

In [5]:
add_error = True
error_rate = 1e-2

num_layout_list = [2**i for i in range(8)]
filepath_list = [os.path.join(os.getcwd(), "data", f"layout_{num_layout}.pickle") for num_layout in num_layout_list]

save_result(filepath_list, add_error=add_error, error_rate=error_rate, do_print=True, save_filepath=os.path.join(os.getcwd(), "data", "accuracy", f"error_rate_{str(error_rate)}.json"))

File: e:\Research\quantum-computing-power-side-channel-attacks\experiments\user_circuit_identification\data\layout_1.pickle
Power Timeseries: 1.0
Total Energy: 0.7419354838709677
Mean Power: 0.6451612903225806
Duration: 0.9354838709677419


File: e:\Research\quantum-computing-power-side-channel-attacks\experiments\user_circuit_identification\data\layout_2.pickle
Power Timeseries: 1.0
Total Energy: 0.7580645161290323
Mean Power: 0.5
Duration: 0.7419354838709677


File: e:\Research\quantum-computing-power-side-channel-attacks\experiments\user_circuit_identification\data\layout_4.pickle
Power Timeseries: 1.0
Total Energy: 0.6854838709677419
Mean Power: 0.27419354838709675
Duration: 0.6532258064516129


File: e:\Research\quantum-computing-power-side-channel-attacks\experiments\user_circuit_identification\data\layout_8.pickle
Power Timeseries: 0.9717741935483871
Total Energy: 0.532258064516129
Mean Power: 0.2217741935483871
Duration: 0.42338709677419356


File: e:\Research\quantum-computing